# Reverse engineer phoneme mapping between PMSP to Jason Zevin (JZ)
- Due to dialact different, abandoned this approach

In [ ]:
%load_ext lab_black
import pandas as pd

I have manually input PMSP Appendix 3: Regularizations of Taraban and McClelland’s (1987) Exception Words in the file mapping_PSPM_JZ.csv

In [ ]:
file = '../common/patterns/mapping_PMSP_JZ.csv'
df = pd.read_csv(file)
df

Strip padding tokens

In [ ]:
df['jz_pho'] = df.dict_p.map(lambda x: x.strip('_ '))
df['pmsp_pho'] = df.Correct_PMSP.map(lambda x: x.strip('/ '))

Check length

In [ ]:
df['jz_len'] = df.jz_pho.map(lambda x: len(x))
df['pmsp_len'] = df.pmsp_pho.map(lambda x: len(x))
print('Item(s) that do not match in length of phoneme:')
df.loc[df.jz_len != df.pmsp_len, :]

Dropping the word "watch" for now...

In [ ]:
df = df.loc[df.index != 6, :]
df.reset_index(inplace=True)

Create letter (phoneme) pairs in all words
- the first letter in a pair is from PMSP, 
- the 2nd letter is from JZ

In [ ]:
pairs = []
for ipho, phoneme in enumerate(df.pmsp_pho):
    for ilet, letter in enumerate(phoneme):
        pairs.append(letter + df.jz_pho[ipho][ilet])

print('Unique phonoeme pairs in all words:')
set(pairs)

These phoneme are duplicated:
- A is duplicated: AE, Ae
- a is duplicated: a@, aa
- u is duplicated: uU, u^

### Mapping Plaut 96 to JZ

In [ ]:
plaut96 = pd.read_csv(
    '../common/patterns/Plaut96.csv',
    header=None,
    na_filter=False,
    names=['word', 'wf', 'wordlen', 'pho', 'pholen', 'what_is_that']
)
plaut96

In [ ]:
plaut = pd.read_csv(
    '../common/patterns/Plaut96.csv',
    header=None,
    na_filter=False,
    names=['word', 'wf', 'wordlen', 'pho', 'pholen', 'what_is_that']
)

jz = pd.read_csv(
    '../common/patterns/6ktraining_v2.dict',
    sep='\t',
    header=None,
    names=['word', 'ort', 'pho', 'wf'],
    na_filter=
    False  # Bug fix: incorrectly treated null as missing value in the corpus
)

In [ ]:
olap = jz.loc[jz.word.isin(plaut.word), :]
olap = olap.loc[:, ['word', 'pho']]
olap.columns = ['word', 'pho_jz']

olap = olap.merge(plaut, on='word')
olap = olap.loc[:, ['word', 'pho_jz', 'pho']]
olap.columns = ['word', 'pho_jz', 'pho_plaut']
olap['pho_jz'] = olap.pho_jz.str.strip('_ ')

### Check len differences...

In [ ]:
olap['jz_len'] = olap.pho_jz.map(lambda x: len(x))
olap['plaut_len'] = olap.pho_plaut.map(lambda x: len(x))
print('Item(s) that do not match in length of phoneme:')
tmp = olap.loc[olap.jz_len != olap.plaut_len, :]
tmp.to_csv('tmp.csv')
tmp

In [ ]:
olap = olap.loc[olap.jz_len == olap.plaut_len, :]
olap.reset_index(inplace=True)

In [ ]:
olap

In [ ]:
jz = pd.read_csv(
    '../common/patterns/6ktraining_v2.dict',
    sep='\t',
    header=None,
    names=['word', 'ort', 'pho', 'wf'],
    na_filter=
    False  # Bug fix: incorrectly treated null as missing value in the corpus
)